In [40]:
!pip install ortools
!python -m pip install -i https://pypi.gurobi.com gurobipy
!python -m pip install cplex

Looking in indexes: https://pypi.gurobi.com


In [8]:
#Patrones y Residuos
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 11 20:27:41 2023

@author: Ruth
"""
import numpy as np

#Generar todos los patrones posibles
def f_sums(v, n): #encontrar todas las listas de sumas
    t=len(v)
    def fsum(p, c):  #encontrar las sumas
        s=sum(c)
        if s<=n:
            ca.append(c[:]) #generando una copia superficial
        if s>n or p>t:
            return
        #bracktrack/ busqueda
        for i in range(p,t):
            k=v[i]
            if s+k <=n:
                c.append(k)
                fsum(i,c)
                c.pop() #elimine el numero que se ha metido
    ca= []
    ca.sort() #ordenar el vector
    fsum(0,[])
    return ca

#generar el vector de residuo
def residuo(combinaciones,l):
    n=len(combinaciones)
    r=np.zeros([n])
    for i in range(n):
        s=sum(combinaciones[i])
        r[i]=l-s
    '''for j in range(n):
        print("Combinacion", combinaciones[j], "residuo", r[j])'''
    return r

#contabilizar los elementos de los patrones
def aij(w,z):

    n=len(z)


    m=len(w)
    v=np.zeros([m,n])

    for i in range(m):

        t=len(w[i])

        for j in range(t):
            if w[i][j]==0:
                break
            if w[i][j]==z[0]:
                v[i][0]+=1
            elif w[i][j]==z[1]:
                v[i][1]+=1
            elif w[i][j]==z[2]:
                v[i][2]+=1
            elif w[i][j]==z[3]:
                v[i][3]+=1
    return v

z=[14,31,36,45]
w=f_sums([14,31,36,45],100)
r=residuo(w, 100)
patrones=aij(w,z)

print(len(r))
#print(f_sums([14,31,36,45],100))
print(aij(w,z))
#print(f_sums([1,5,3,2],6))
#print(f_sums([14,31,36,45],100))


38
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [2. 0. 0. 0.]
 [3. 0. 0. 0.]
 [4. 0. 0. 0.]
 [5. 0. 0. 0.]
 [6. 0. 0. 0.]
 [7. 0. 0. 0.]
 [4. 1. 0. 0.]
 [4. 0. 1. 0.]
 [3. 1. 0. 0.]
 [3. 0. 1. 0.]
 [3. 0. 0. 1.]
 [2. 1. 0. 0.]
 [2. 2. 0. 0.]
 [2. 1. 1. 0.]
 [2. 0. 1. 0.]
 [2. 0. 2. 0.]
 [2. 0. 0. 1.]
 [1. 1. 0. 0.]
 [1. 2. 0. 0.]
 [1. 1. 1. 0.]
 [1. 1. 0. 1.]
 [1. 0. 1. 0.]
 [1. 0. 2. 0.]
 [1. 0. 1. 1.]
 [1. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 2. 0. 0.]
 [0. 3. 0. 0.]
 [0. 2. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 2. 0.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 2.]]


In [41]:
#Modelo con Gurobi
from gurobipy import *
import numpy as np

m = Model('Cortes Basado en Residuos')

# Matriz de Patrones
Patrones = aij(w,z)

# Numero de patrones
n_patrones = len(Patrones)

# Número de Productos
n_productos = 4

# Demandas a satisfacer por producto
Demanda=[211,395,610,97]

# Indices para los patrones
I = np.arange(0,n_patrones)

# Creación de las variables de decisión:
x = m.addVars(I, vtype = GRB.INTEGER, name = "x")

# Restricción de Demanda por cada producto
m.addConstrs(quicksum( Patrones[ patron , producto]*x[patron] for patron in I )
                      >= Demanda[producto] for producto in range(n_productos))

# Funcion objetivo
Funcion_objetivo = quicksum( r[patron] * x[patron] for patron in I )

m.setObjective(Funcion_objetivo, GRB.MINIMIZE)

m.optimize()

print("# ------------- Solución con GUROBI -------------------------- #")

for patron in I:
  if x[patron].X != 0:
    print("Se deben cortar del patrón", patron,
          "esta cantidad de productos", x[patron].X)

valor_objetivo = m.objVal
print("Valor de la función objetivo en Gurobi:", valor_objetivo)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 4 rows, 38 columns and 64 nonzeros
Model fingerprint: 0xe6dca420
Variable types: 0 continuous, 38 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 6e+02]
Found heuristic solution: objective 9480.0000000
Presolve removed 2 rows and 34 columns
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 5 nonzeros
Found heuristic solution: objective 881.0000000
Variable types: 0 continuous, 4 integer (0 binary)

Root relaxation: objective 8.800000e+02, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

In [47]:
from ortools.linear_solver import pywraplp
import numpy as np

# Crear un solucionador MIP de Gurobi
solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    exit()

# Definir el nombre del modelo
model_name = "Cortes_Basado_en_Residuos"

# Matriz de Patrones
Patrones = aij(w, z)

# Número de patrones
n_patrones = len(Patrones)

# Número de Productos
n_productos = 4

# Demandas a satisfacer por producto
Demanda = [211, 395, 610, 97]

# Indices para los patrones
I = np.arange(0, n_patrones)

# Crear variables de decisión
x = {}
for patron in I:
    x[patron] = solver.IntVar(0, solver.infinity(), f'x[{patron}]')

# Restricción de Demanda por cada producto
for producto in range(n_productos):
    solver.Add(solver.Sum(Patrones[patron][producto] * x[patron] for patron in I) >= Demanda[producto])

# Función objetivo
Funcion_objetivo = solver.Sum(r[patron] * x[patron] for patron in I)

solver.Minimize(Funcion_objetivo)

solver.Solve()

print("# ------------- Solución con OR-Tools -------------------------- #")

valor_objetivo = solver.Objective().Value()
print("Valor de la función objetivo en OR-Tools:", valor_objetivo)

for patron in I:
    if x[patron].solution_value() != 0:
        print(f"Se deben cortar del patrón {patron} esta cantidad de productos {x[patron].solution_value()}")


# ------------- Solución con OR-Tools -------------------------- #
Valor de la función objetivo en OR-Tools: 881.0
Se deben cortar del patrón 17 esta cantidad de productos 1e+20
Se deben cortar del patrón 25 esta cantidad de productos 97.0
Se deben cortar del patrón 30 esta cantidad de productos 198.0


In [43]:
import cplex

# Crear un objeto CPLEX
problem = cplex.Cplex()

# Definir la función objetivo como minimización
problem.objective.set_sense(problem.objective.sense.minimize)

# Matriz de Patrones
Patrones = aij(w, z)

# Número de patrones
n_patrones = len(Patrones)

# Número de Productos
n_productos = 4

# Demandas a satisfacer por producto
Demanda = [211, 395, 610, 97]

# Indices para los patrones
I = range(n_patrones)

# Crear variables de decisión como enteras
x = problem.variables.add(names=[f'x_{patron}' for patron in I], lb=[0] * n_patrones, ub=[cplex.infinity] * n_patrones, types=['I'] * n_patrones)

# Restricción de Demanda por cada producto
for producto in range(n_productos):
    coefficients = [Patrones[patron][producto] for patron in I]
    problem.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=I, val=coefficients)],
        senses=['G'],
        rhs=[Demanda[producto]]
    )

# Función objetivo
coefficients = [r[patron] for patron in I]
problem.objective.set_linear(enumerate(coefficients))

# Resolver el problema
problem.solve()

# Imprimir la solución
print("\n # ------------- Solución con CPLEX -------------------------- #")

valor_objetivo = problem.solution.get_objective_value()
print("Valor de la función objetivo en CPLEX:", valor_objetivo)

for patron, var in enumerate(x):
    if problem.solution.get_values(var) > 0:
        print(f"Se deben cortar del patrón {patron}, esta cantidad de productos: {problem.solution.get_values(var)}")


Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 29422.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 2 rows and 26 columns.
MIP Presolve added 2 rows and 2 columns.
Reduced MIP has 4 rows, 14 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.03 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 2 rows and 2 columns.
MIP Presolve added 2 rows and 2 columns.
Reduced MIP has 4 rows, 14 columns, and 26 nonzeros.
Reduced MIP has 0 binaries, 14 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node